<a href="https://colab.research.google.com/github/juelha/IANNWTF/blob/sabine/hw08_Sabine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras

In [ ]:
# dataset and prepossing
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

shape_train = x_train.shape
shape_test = x_test.shape

x_train = tf.data.Dataset.from_tensor_slices(x_train)
x_test = tf.data.Dataset.from_tensor_slices(x_test)
print(x_train)

def preprocessing(tensor):
  tensor = tensor.map(lambda images: (tf.cast(images, tf.float32) / 255.)) # normalise
  tensor = tensor.map(lambda images: (tf.expand_dims(images, -1))) # make 3D in case
  tensor = tensor.map(lambda images: (tf.add(images, tf.random.normal((28,28,1), mean=0, stddev=1)))) # add noise with random 28,28 matrix 
  tensor = tensor.map(lambda images: (tf.clip_by_value(images, clip_value_min=0, clip_value_max=1))) # keep values normalised by clipping
  #tensor = tensor.map(lambda images: (tf.cast(images, tf.uint8)))
  print(tensor)
  tensor = tensor.shuffle(1000)
  tensor = tensor.batch(32)
  tensor = tensor.prefetch(20)
  return tensor


train_dataset = x_train.apply(preprocessing)
test_dataset = x_test.apply(preprocessing)

for image in train_dataset:
  train_data = image
  break

print(train_data.shape)


<TensorSliceDataset shapes: (28, 28), types: tf.uint8>
<MapDataset shapes: (28, 28, 1), types: tf.float32>
<MapDataset shapes: (28, 28, 1), types: tf.float32>
(32, 28, 28, 1)


#2.1 Convolutional Autoencoder

• The Autoencoder should consist of an encoder and a decoder, which can
be called independently. 6
• Encoder: The encoder should reduce the size of feature maps like a CNN.78
At the end of the encoder, flatten the feature maps and use a dense layer
to produce an embedding of a certain size. 9
• Decoder: The decoder takes the embedding from the encoder as input.
Use a dense layer to restore the dimensionality of the flattened feature
maps from the encoder and reshape the resulting vector into feature maps
again. Use upsampling or transposed convolutions to mirror your encoder.
As an output layer, use a convolutional layer with one filter and sigmoid
activation to produce an output image.


In [ ]:
class Encoder(tf.keras.Model): 
  def __init__(self, latent_dim=10): 
    super(Encoder, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(28, 28))
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(7*7*64, activation='relu') # LOOK HERE JAY: Use a dense layer to force a tensor shape that in the encoder that can be reshaped in the decoder!
    self.dense2 = tf.keras.layers.Dense(latent_dim)

  @tf.function
  def call(self, input):
    x = self.conv1(input)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    return x

In [ ]:
class Decoder(tf.keras.Model): 
  def __init__(self):
    super(Decoder, self).__init__()
    self.dense = tf.keras.layers.Dense(7*7*64, activation='relu')
    self.reshape = tf.keras.layers.Reshape((7,7,64))
    self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')
    self.out = tf.keras.layers.Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='relu')

  @tf.function
  def call(self, input): 
    x = self.dense(input)
    x = self.reshape(x)
    x = self.conv1(x)
    x = self.out(x)
    return x


In [ ]:
class Autoencoder(tf.keras.Model): 
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = Encoder()
    self.decoder = Decoder()

  @tf.function
  def call(self, input): 
    x = self.encoder(input)
    x = self.decoder(x)
    return x


In [ ]:
model = Autoencoder()
out = model(train_data)